# 財報分析指標

* 營業毛利 = 營業收入 - 營業成本
* 毛利率(Profit margin) = 營業毛利 / 營業收入 x 100%
* 營業利益 = 營業毛利 - 營業費用
* 營業利益率(營益率、Operating Margin)：營業利益 / 營業收入 x 100%
* 稅前純益 = 營業利益 + 業外收支
* 純益率(Pre-Tax Income Margin) = 稅前純益 / 營業收入 x 100%

In [51]:
import sqlite3, os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
conn = sqlite3.connect(os.path.join('data', 'fs.db'))

In [1]:
sid = input('請輸入股票代碼：')

query = "select * from income where sid="+sid
df = pd.read_sql(query, conn, index_col=['sid','year','season'])

if not len(df):
    print('無此股票資料！')
else:
    print('資料已讀取！')

In [2]:
df['營業毛利'] = df['營業收入合計'] - df['營業成本合計']
df['營業利益'] = df['營業毛利'] - df['營業費用合計']
df['稅前純益'] = df['營業利益'] + df['營業外收入及支出合計']
df['毛利率'] = round(df['營業毛利'] / df['營業收入合計'] * 100, 2)
df['營益率'] = round(df['營業利益'] / df['營業收入合計'] * 100, 2)
df['純益率'] = round(df['稅前純益'] / df['營業收入合計'] * 100, 2)

df.head(10)

In [3]:
# 繪製財報三率圖

plt.rcParams['figure.figsize'] = (12, 6) 
df['毛利率'].plot()
df['營益率'].plot()
df['純益率'].plot()
plt.legend()

In [4]:
df = df.reset_index()
df.head(10)

In [5]:
# 建立一個 figure 和一個 subplot
fig, ax = plt.subplots(figsize=(15,7))
df.groupby('year').plot(x='season', y='毛利率', ax=ax)
plt.title('毛利率', fontsize=18)
plt.legend(df.year.unique())

fig, ax = plt.subplots(figsize=(15,7))
df.groupby('year').plot(x='season', y='營益率', ax=ax)
plt.title('營益率', fontsize=18)
plt.legend(df.year.unique())

fig, ax = plt.subplots(figsize=(15,7))
df.groupby('year').plot(x='season', y='純益率', ax=ax)
plt.title('純益率', fontsize=18)
plt.legend(df.year.unique())

In [6]:
# 建立一個 figure 和 3 個 subplot

plt.rcParams['figure.figsize'] = (10, 8)
fig, axes = plt.subplots(nrows=3, sharex=True)
df.groupby('year').plot(x='season', y='毛利率', ax=axes[0], legend=None)
axes[0].set_title('毛利率', fontsize=14)
#axes[0].legend(df.year.unique())
df.groupby('year').plot(x='season', y='營益率', ax=axes[1], legend=None)
axes[1].set_title('營益率', fontsize=14)
#axes[1].legend(df.year.unique())
df.groupby('year').plot(x='season', y='純益率', ax=axes[2])
axes[2].set_title('純益率', fontsize=14)
axes[2].legend(df.year.unique())
plt.tight_layout()

# 本益比計算

本益比(Price-to-Earning ratio)：股價／EPS

In [85]:
conn1 = sqlite3.connect(os.path.join('data', 'data.db'))

In [7]:
# 讀取股價資料

query = "select * from price where sid="+sid+" and date between datetime('2019-01-01') and datetime('2019-12-31')"
price = pd.read_sql(query, conn1, index_col='date')

price

In [8]:
# 尋找全年最高價、最低價與平均價

print('全年最高價：', price['收盤價'].max())
print('全年最低價：', price['收盤價'].min())
print('全年平均價：', round(price['收盤價'].mean(),2))

In [9]:
# 讀取 eps 資料

query = "select * from income where sid="+sid+" and year='108'"
eps = pd.read_sql(query, conn, index_col='season')

eps

In [10]:
# 計算全年 EPS

total = eps.loc['04']['基本每股盈餘']
# eps.基本每股盈餘
# total = eps.基本每股盈餘.sum()
total

In [11]:
# 計算本益比

print('最低本益比：', round(price['收盤價'].min()/total,2))
print('最高本益比：', round(price['收盤價'].max()/total,2))
print('平均本益比：', round(price['收盤價'].mean()/total,2))

# 股價淨值比 - 資產負債表

* 股價淨值比(Price-Book Ratio)：股價 / 每股淨值
* 每股淨值 = 淨值 / 流通在外股數
* 流通在外股數 = 普通股股本 / 10
* 淨值 = 總資產 – 總負債 = 股東權益 (歸屬於母公司業主之權益合計)

In [12]:
# 讀取資產負債表資料
query = "select * from bs where sid="+sid+" and year='108'"
df = pd.read_sql(query, conn, index_col='season')
print(df)

# 讀取股價資料
query = "select * from price where sid="+sid+" and date between datetime('2019-12-20') and datetime('2019-12-31')"
price = pd.read_sql(query, conn1, index_col='date')
price

In [13]:
股本 = df['普通股股本']['04']
股東權益 = df['歸屬於母公司業主之權益合計']['04']
股價 = price.iloc[-1]['收盤價']
print('股本 =', 股本)
print('股東權益 =', 股東權益)
print('股價 =', 股價)

In [14]:
# 計算股價淨值比

PB = round(股價 / (股東權益 / (股本 / 10)), 2)
print('股價淨值比 =', PB)

# 找出某個特定欄位資料

合約負債：https://www.youtube.com/watch?v=VcQzpecU9q8

In [15]:
# 查詢合約負債資料

query = "select sid, year, season, 合約負債－流動 from bs"
#query = "select sid, year, season, 合約負債－流動 from bs where 合約負債－流動 > 0"
df = pd.read_sql(query, conn, index_col=['sid', 'year','season'])

df